# Hands-on!

Nessa prática, sugerimos alguns pequenos exemplos para você implementar sobre o Spark.

## Estimar o Pi

Existe um algoritmo para estimar o Pi com números radômicos. Implemente-o sobre o Spark.

Descrição do algoritmo: http://www.eveandersson.com/pi/monte-carlo-circle

Implementação EM PYTHON (__não sobre o SPARK__): http://www.stealthcopter.com/blog/2009/09/python-calculating-pi-using-random-numbers/

O númer de pontos deve ser 100000 (cem mill) vezes o número mínimo de partições padrão do seu SparkContext (`sc.defaultMinPartitions`). Esses pontos devem ser selecionados aleatóriamente na etapa de map (ver observações).

Observações: use as funções __map__ (para mapear as ocorrêncas em `0` ou `1`, significando `1` quando o ponto aleatório cair dentro do círculo e `0` quando o contrário) e __reduce__ (para sumar as ocorrências).

In [19]:
from random import *
from math import sqrt
inside=0
n=1000
for i in range(0,n):
    x=random()
    y=random()
    if sqrt(x*x+y*y)<=1:
        inside+=1
pi=4*inside/n

print(pi)

3.164


In [26]:
from random import *
from math import sqrt
def soma(a,b): return a+b
def area(x,y):return 1 if sqrt(x*x+y*y)<=1 else  0
def mapfunction(z):
    x=random()
    y=random()
    return area(x,y)


In [27]:
nummin = sc.defaultMinPartitions * 10000
data = sc.parallelize(range(1, nummin))
res1 = data.map(mapfunction)

In [28]:
print(res1.reduce(soma))

15767


In [30]:
pispark = 4*(res1.reduce(soma))/(res1.count())
print(pispark)

3.153557677883894


## Filtragem de Primos

Dado uma sequência de números de `1` a `1000000`, filtre somente os primos dessa sequência.

In [37]:
def primes( n ):
# i sera nosso divisor inicial
    i = 1;
# j sera nosso contador de ocorrências
    j = 0;
#Nenhum numero real vai ser divisivel por um numero maior do que sua metade
    n1 = (n/2);

    while (i <= n):

       if (n % i==0):
          i = i+1;
          j = j+1;

       if (i>=n1):
        # damos a i, o valor da variavel entrada, pois o próximo divisor sera o próprio número
          i = n;
          i = i+1;
          j = j+1;

       else:
          i = i+1;
    if(j==2):
        return True
    else: 
        return False

In [39]:
data = sc.parallelize(range(1, 1000000))
res = data.filter(primes)
print(res.collect())

## Municípios do Brasil

Dado o dataset `mucipios_do_Brasil.csv`, faça duas operações com ele:

1. Monte uma lista dos municípios por estado.
2. Conte quantos municípios há em cada estado.

Dicas: use as operações groupByKey e reduceByKey, não faça um count na lista da operação 1. 

In [46]:
lines = sc.textFile("municipios_do_Brasil.csv")

data = lines.map(lambda line: line.split(","))

print(data.take(2))

[['uf', 'cidade', 'lat', 'lng', 'cond'], ['AC', 'Acrelândia', '-9.825808', '-66.897166', 'false']]


In [47]:
data.groupByKey(lambda uf,cidade)

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/tmp/kernel-PySpark-74e71d1e-d799-47f4-8575-d4ff8340c44f/pyspark_runner.py", line 188, in <module>
    ast_parsed = ast.parse(final_code)
  File "/home/bruno/miniconda3/envs/spark/lib/python3.6/ast.py", line 35, in parse
    return compile(source, filename, mode, PyCF_ONLY_AST)
  File "<unknown>", line 1
    data.groupByKey(lambda uf,cidade)
                                    ^
SyntaxError: invalid syntax

StackTrace: org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:163)
org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:163)
scala.Option.foreach(Option.scala:257)
org.apache.toree.interpreter.broker.BrokerState.markFailure(BrokerState.scala:162)
sun.reflect.GeneratedMethodAccessor66.invoke(Unknown Source)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43

In [43]:
df.groupByKey("uf")

Name: org.apache.toree.interpreter.broker.BrokerException
Message: Traceback (most recent call last):
  File "/tmp/kernel-PySpark-74e71d1e-d799-47f4-8575-d4ff8340c44f/pyspark_runner.py", line 194, in <module>
    eval(compiled_code)
  File "<string>", line 1, in <module>
  File "/home/bruno/spark-2.2.0-bin-hadoop2.7/python/pyspark/sql/dataframe.py", line 1020, in __getattr__
    "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
AttributeError: 'DataFrame' object has no attribute 'groupByKey'

StackTrace: org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:163)
org.apache.toree.interpreter.broker.BrokerState$$anonfun$markFailure$1.apply(BrokerState.scala:163)
scala.Option.foreach(Option.scala:257)
org.apache.toree.interpreter.broker.BrokerState.markFailure(BrokerState.scala:162)
sun.reflect.GeneratedMethodAccessor66.invoke(Unknown Source)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)


## Word Count Memória Postumas de Brás Cubas

Memórias Póstumas de Brás Cubas é um romance escrito por Machado de Assis, desenvolvido em princípio como folhetim, de março a dezembro de 1880, na Revista Brasileira, para, no ano seguinte, ser publicado como livro, pela então Tipografia Nacional.

A obra retrata a escravidão, as classes sociais, o cientificismo e o positivismo da época. Dada essas informações, será que conseguimos idenficar essas características pelas palavras mais utilizadas em sua obra?

Utilizando o dataset `Machado-de-Assis-Memorias-Postumas.txt`, faça um word count e encontre as palavras mais utilizadas por Machado de Assis em sua obra. Não esqueça de utilizar `stopwords.pt` para remover as `stop words`!